# 步驟一、找出想抓的新聞類別

吃到飽 https://news.tvbs.com.tw/realtime
- 若覺得資料不夠多，利用日期去調整出以前的新聞(大約1周上下的區間)

# 步驟二、從網頁原始碼觀察是何種類型

In [63]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd

# 步驟三、先以一頁形式觀察其規律

In [64]:
url = "https://news.tvbs.com.tw/realtime/news/2021-12-03"
response = requests.get(url, verify = False)
response.encoding = "utf-8"
html = BeautifulSoup(response.text)
html

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<!DOCTYPE html>
<html lang="zh-Hant-TW">
<head>
<meta charset="utf-8"/>
<title>即時新聞│TVBS新聞網</title>
<meta content="即時新聞,新聞,氣象,focus,食尚,科技,新奇,健康,星聞,影劇,體育,焦點,民生消費,政經,全球,議題,直播,T談談,民調,NEWS,TVBS NEWS" name="Keywords"/>
<meta content="最新畫面、即時焦點、完整公正的新聞內容就在TVBS新聞網，您最值得信賴的媒體。" name="description"/>
<link href="https://cc.tvbs.com.tw/2017news/prd/images/favicon.ico?v=005" rel="icon" type="image/icon"/>
<!--Device Web-->
<meta content="width=device-width" name="viewport"/>
<!--Device Web ed-->
<link href="https://cc.tvbs.com.tw/2017news/prd/depoly/all_new_realtime_main_v01.css?v=005" media="screen" rel="stylesheet" type="text/css"/>
<script src="https://cc.tvbs.com.tw/2017news/prd/depoly/all_new_realtime_main_v01.js?v=005" type="text/javascript"></script>
<script async="" src="https://securepubads.g.doubleclick.net/tag/js/gpt.js"></script><script>window.googletag=window.googletag||{cmd:[]};var news_tvbs_com_tw_pc_index_top;var news_tvbs_com_tw_pc_index_list1;var news_tvbs_com_tw_pc_index_list2;va

In [65]:
table = {
    "Headline" : [],
    "Link" : [],
    "Date" : [],
    "Source" : [],
    "Category" : [],
}

In [66]:
newslists = html.find("div", class_ = "news_list").select_one("div.list").find("ul").find_all("li")
# newslists
for newslist in newslists:
    try:
        date_not_split = newslist.find("a").find("div", class_ = "img").find("img")["data-original"]
        date = date_not_split.split("/")[-4:-1]
        date = "/".join(date)
        table["Date"].append(date)
        print("日期:", date)
        headline = newslist.find("h2", class_ = "txt").text
        table["Headline"].append(headline)
        print("標題:", headline)
        link = newslist.find("a")["href"]
        table["Link"].append(link)
        print("連結:", link)
        category = link.split("/")[1]
        table["Category"].append(category)
        print("類別:", category)
        source = "TVBS"
        table["Source"].append(source)
        print("來源:", source)
        print("=" * 25)
    except AttributeError:
        print("fail")

日期: 2021/09/03
標題: 打第3劑防魔王株？　專家揭6類、9族群「最佳組合」
連結: /life/1651411
類別: life
來源: TVBS
日期: 2021/05/11
標題: 孩子回來了？婦流產2次又懷孕　超音波浮出「詭異人臉」
連結: /world/1651394
類別: world
來源: TVBS
日期: 2021/11/30
標題: 快訊／台鐵頭城-龜山「斷軌0.5cm」　降速影響5列次
連結: /local/1651409
類別: local
來源: TVBS
fail
日期: 2021/12/03
標題: 罷昶案成立！1/9投票　林昶佐：正面迎戰
連結: /politics/1651400
類別: politics
來源: TVBS
日期: 2021/05/08
標題: 咖啡控注意！　專家：「這款」連喝1個月胖2.2公斤
連結: /life/1651407
類別: life
來源: TVBS
日期: 2021/12/03
標題: 新北耶誕城開城！　「樂高」300秒光雕秀超吸睛
連結: /life/1651396
類別: life
來源: TVBS
日期: 2021/10/17
標題: 天冷吃鍋睜大眼！不想吃假雪花牛　專家授外觀分辨訣竅
連結: /life/1651399
類別: life
來源: TVBS
fail
日期: 2021/12/03
標題: 想去玩！2022報復旅遊潮？未來旅行趨勢報告出爐　寵物友善、環保、在地成旅遊首選
連結: /life/1651260
類別: life
來源: TVBS
日期: 2021/11/18
標題: 拍成人片賺翻！才買百萬特斯拉　米砂「千萬豪宅」驚曝光
連結: /entertainment/1651402
類別: entertainment
來源: TVBS
日期: 2021/12/03
標題: 賣方市場正熱門　實價登錄「助漲」預售屋價格
連結: /life/1651319
類別: life
來源: TVBS
日期: 2021/05/28
標題: Omicron掀全球恐慌　世衛：尚未接獲死亡通報
連結: /world/1651360
類別: world
來源: TVBS
fail
日期: 2021/08/27
標題: 財神眷顧！大樂透頭獎2億元　「1注獨得」獎落這縣市
連結: /life/

# 步驟四、套用步驟三的程式到其他頁數，並且把資料存到table

# 步驟五、把table用pandas顯示

In [67]:
# 解決欄位資料有沒抓成功的資料而無法轉成df顯示的問題
df = pd.DataFrame.from_dict(table, orient = "index")
df = df.transpose() # 將df轉置
df.tail(5) # 取倒數5筆資料
# df.head()

,Headline,Link,Date,Source,Category
449,別急著起鍋！煎蛋多1撇步香氣秒提升 老饕激推：超下飯,/life/1650107,2021/12/02,TVBS,life
450,大學那些科系夯爆卻很慘？鄉民點3系所：沒證照就廢掉,/life/1649554,2021/11/19,TVBS,life
451,願意負責被拒！高三男搞大肚 女友家長「索100萬」急求助,/life/1650367,2021/12/02,TVBS,life
452,擦藥也沒用！青春期滿臉「痘」花怎解 內行教養皮膚關鍵,/life/1649950,2021/12/02,TVBS,life
453,好市多廁所新設計！他讚「衛生」反掀論戰：旁邊沾糞超噁,/life/1649886,2021/12/02,TVBS,life


# 步驟六、存檔

In [68]:
# 存成csv檔
# url = "https://www.chinatimes.com/society/total?page=1&chdtv"
# fn = url_.split("/")[3]
path = "/content/drive/Shareddrives/Media Framing/datasets/tvbs/"
if not os.path.exists(path):
    os.makedirs(path)

df.to_csv(path + "tvbs_2021-12-03.csv", encoding = "utf-8", index = False)

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 檢查df檔案

In [70]:
df_ = pd.read_csv("/content/drive/Shareddrives/Media Framing/datasets/tvbs/tvbs_2021-12-03.csv")
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  454 non-null    object
 1   Link      454 non-null    object
 2   Date      454 non-null    object
 3   Source    454 non-null    object
 4   Category  454 non-null    object
dtypes: object(5)
memory usage: 17.9+ KB
